# Imports

In [ ]:
!pip install transformers datasets sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Uploading Data

In [ ]:
train_df = pd.read_csv("train.txt", sep=';', header=None, names=['text', 'label'])
test_df = pd.read_csv("test.txt", sep=';', header=None, names=['text', 'label'])
val_df = pd.read_csv("val.txt", sep=';', header=None, names=['text', 'label'])


coverting text to list and giving them numbers like 0 1 2 3

In [ ]:
labels = train_df['label'].unique().tolist()
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}


applying thos number in real data sets

In [ ]:
train_df['label'] = train_df['label'].map(label2id)
val_df['label'] = val_df['label'].map(label2id)
test_df['label'] = test_df['label'].map(label2id)

In [ ]:
train_df

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,5
15998,i feel like this was such a rude comment and i...,1


In [ ]:
print(label2id)

{'sadness': 0, 'anger': 1, 'love': 2, 'surprise': 3, 'fear': 4, 'joy': 5}


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
print(train_df.head())

                                                text  label
0                            i didnt feel humiliated      0
1  i can go from feeling so hopeless to so damned...      0
2   im grabbing a minute to post i feel greedy wrong      1
3  i am ever feeling nostalgic about the fireplac...      2
4                               i am feeling grouchy      1


#Tokenization

🧠 What is Tokenization?
LLMs like BERT don’t understand plain text like:


"I feel happy today"


They need it converted into numbers (tokens) that represent words or subwords. This process is called tokenization.

We use a special tool for that: a tokenizer — and Hugging Face provides one ready for BERT.



In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Load Pretrained BERT Model

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(

                                                           model_name,
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id

)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**🔍 What’s Happening Here?**



🔹 AutoModelForSequenceClassification:
This loads a BERT model with a final layer designed for classification tasks (like emotion detection).

It will return probabilities for each emotion (joy, anger, etc.).

🔹 num_labels=len(labels):
If you have 6 emotions (joy, sadness, fear, etc.), it tells the model: “Output 6 probabilities.”



# Define Evaluation Metric (Accuracy)

In [ ]:
def compute_matrix(p):
  preds = np.argmax(p.predictions, axis = 1)
  return {
      "accuracy": accuracy_score(p.label_ids,preds)
  }

#  Set Up Training Arguments

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",             # where to save model and checkpoints
    eval_strategy="epoch",              # run evaluation after each epoch
    save_strategy="epoch",              # save model after each epoch
    learning_rate=2e-5,                 # how fast the model learns (2e-5 = 0.00002)
    per_device_train_batch_size=16,     # number of examples in one training batch
    per_device_eval_batch_size=16,      # number of examples in one evaluation batch
    num_train_epochs=3,                 # how many times to go through the full dataset
    weight_decay=0.01,                  # regularization to prevent overfitting
    logging_dir="./logs",               # where to save training logs
    logging_steps=10,                   # log every 10 steps
    load_best_model_at_end=True,        # restore best model based on eval metric
    metric_for_best_model="accuracy"    # choose best model by accuracy
)

#  Train Your BERT Model!

In [ ]:
from transformers import Trainer

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_matrix,
    tokenizer=tokenizer
)

trainer.train()

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/tmp/ipython-input-77901806.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: reiki2607 (reiki2607-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

# Evaluate Your Model on Test Data

In [ ]:
results = trainer.evaluate(test_dataset)
print("Test Accuracy:", results["eval_accuracy"])


# See a Detailed Classification Report

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)

print(classification_report(test_dataset["label"], pred_labels, target_names=labels))
